In [1]:
import numpy as np
import pandas as pd
import torch
import torchtext
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.functional import numericalize_tokens_from_iterator
from torch.nn.utils.rnn import pad_sequence

import logging
import os
import warnings
warnings.filterwarnings("ignore")

# Due to warning when initializing the "spacy" tokenizer
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # disable tensorflow logging
logging.getLogger('tensorflow').disabled = True  # disable tensorflow warning messages



In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
df = pd.read_csv('artifacts/train_cleaned.csv')

# limit df
df = df[:10000]
df

,tweet,sentiment,label
0,im getting on borderlands and i will murder yo...,Positive,3
1,I am coming to the borders and I will kill you...,Positive,3
2,im getting on borderlands and i will kill you ...,Positive,3
3,im coming on borderlands and i will murder you...,Positive,3
4,im getting on borderlands 2 and i will murder ...,Positive,3
...,...,...,...
9995,_. . The faster more efficient AMD Zen CPUs in...,Positive,3
9996,_.. The faster more efficient AMD Zen CPUs in ...,Positive,3
9997,_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ ...,Positive,3
9998,_.. The faster memory efficient AMD Zen CPUs i...,Positive,3


In [4]:
df['label'].unique()

array([3, 2, 1, 0], dtype=int64)

In [5]:
df_valid = pd.read_csv('artifacts/valid_cleaned.csv')

df_valid = df_valid[:1000]
df_valid

,tweet,sentiment,label
0,I mentioned on Facebook that I was struggling ...,Irrelevant,0
1,BBC News - Amazon boss Jeff Bezos rejects clai...,Neutral,2
2,@Microsoft Why do I pay for WORD when it funct...,Negative,1
3,"CSGO matchmaking is so full of closet hacking,...",Negative,1
4,Now the President is slapping Americans in the...,Neutral,2
...,...,...,...
995,⭐️ Toronto is the arts and culture capital of ...,Irrelevant,0
996,tHIS IS ACTUALLY A GOOD MOVE TOT BRING MORE VI...,Irrelevant,0
997,Today sucked so it’s time to drink wine n play...,Positive,3
998,Bought a fraction of Microsoft today. Small wins.,Positive,3


In [6]:
df_valid['label'].unique()

array([0, 2, 1, 3], dtype=int64)

In [7]:
tokenizer = get_tokenizer("spacy")

def token_gen(text):
    """
    Tokenizes each sentence in a given text and yields the resulting tokens.

    Args:
        text (list[str]): A list of sentences to tokenize.

    Yields:
        list[str]: The resulting tokens from each sentence.
    """
    for sent in text:
        tokens = tokenizer(sent)
        yield tokens



In [8]:
# assign 2 hyper-parameters
VOCAB_SIZE = 5000
MAX_LENGTH = 100 # to restrict length every sequence 

vocab = build_vocab_from_iterator(token_gen(df['tweet']),specials=["<UNK>"],max_tokens=VOCAB_SIZE)
vocab.set_default_index(vocab["<UNK>"])  ## to handel OOV problem


In [9]:
#print(vocab.get_stoi())

In [10]:
# numericalize tokens from iterator using vocab
sequence = numericalize_tokens_from_iterator(vocab=vocab,iterator=token_gen(df['tweet']))

print('data type is:',type(sequence))

count=0
for ids in sequence:
    print([num for num in ids])
    count+=1
    if count==10:
        break
    


data type is: <class 'generator'>
[44, 157, 238, 14, 99, 7, 44, 84, 2081, 19, 41, 5]
[3, 130, 405, 6, 2, 0, 7, 3, 84, 754, 19, 41, 5]
[44, 157, 238, 14, 99, 7, 44, 84, 754, 19, 41, 5]
[44, 157, 405, 14, 99, 7, 44, 84, 2081, 19, 41, 5]
[44, 157, 238, 14, 99, 53, 7, 44, 84, 2081, 19, 38, 41, 5]
[44, 157, 238, 225, 99, 7, 44, 87, 2081, 19, 41, 5]
[140, 3, 828, 8, 413, 288, 486, 236, 9, 100, 1, 1, 1, 162, 19, 45, 40, 173, 3, 130, 8, 2160, 197, 628, 7, 1231, 11, 70, 10, 24, 240, 431, 1, 140, 3, 797, 6, 182, 433, 8, 0, 9, 24, 292, 1, 1, 635, 11, 2, 933, 1884, 0, 2, 0, 3, 211, 832, 1420, 4, 0]
[140, 3, 828, 8, 1188, 10, 288, 385, 236, 9, 100, 16, 162, 19, 45, 40, 173, 23, 3, 55, 8, 524, 25, 32, 628, 7, 1231, 11, 70, 10, 24, 240, 431, 5, 3, 797, 6, 182, 8, 0, 9, 24, 292, 20, 635, 29, 2, 933, 2702, 955, 6, 2, 0, 3, 211, 832, 498, 100, 4, 321, 12, 0]
[140, 3, 828, 8, 413, 288, 385, 236, 9, 100, 16, 162, 19, 45, 40, 173, 3, 55, 8, 2160, 25, 32, 628, 7, 1231, 11, 70, 10, 24, 240, 431, 1]
[140, 3, 

In [11]:
## check how "numericalize_tokens_from_iterator" works

from torchtext.data.functional import numericalize_tokens_from_iterator

sequence = numericalize_tokens_from_iterator(vocab,["hi how are you", "what is your name?"])
list(next(sequence))

[1753, 44, 31, 1753, 3857, 881, 31, 8, 1022, 846, 31, 576, 3857, 379]

### option 1

```
token_ids = []
for i in range(len(df)):
    token_id = vocab(tokenizer(df['tweet'][i]))
    token_ids.append(token_id)
    
padded_text = pad_sequence([torch.tensor(x) for x in token_ids], batch_first=True, padding_value=0)

padded_text = padded_text[:,:MAX_LENGTH]
padded_text

```

In [12]:
### option 2

# numericalize tokens from iterator using vocab
sequence = numericalize_tokens_from_iterator(vocab=vocab,iterator=token_gen(df['tweet']))

# create a list to store tokenized sequences
token_ids = []
for i in range(len(df)):
    x = list(next(sequence))
    token_ids.append(x)

# valid_token_ids = torch.tensor(valid_token_ids) # this will throw an error, because all sequence are not of same length

# Pad the sequences to the same length along dimension 0
padded_text = pad_sequence([torch.tensor(x) for x in token_ids], batch_first=True, padding_value=0)

# restrict the length of every sequence upto MAX_LENGTH
padded_text = padded_text[:,:MAX_LENGTH]

print(padded_text.shape)
print(padded_text)

torch.Size([10000, 100])
tensor([[ 44, 157, 238,  ...,   0,   0,   0],
        [  3, 130, 405,  ...,   0,   0,   0],
        [ 44, 157, 238,  ...,   0,   0,   0],
        ...,
        [ 51,  51,  51,  ...,  51,  51,  51],
        [ 51,  20,  39,  ...,   0,   0,   0],
        [ 51,  20,  39,  ...,   0,   0,   0]])


In [13]:
len(padded_text)

10000

In [14]:
padded_text[0]

tensor([  44,  157,  238,   14,   99,    7,   44,   84, 2081,   19,   41,    5,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0])

In [15]:
tokenizer('I will see you!')

['I', 'will', 'see', 'you', '!']

In [16]:
## vocab([tokens])

vocab(tokenizer('I will see you!')) ## similar to the 'fit_on_texts()' in tensorflow

[3, 84, 62, 19, 4]

In [17]:
torch.tensor(vocab(tokenizer('I will see you')))

tensor([ 3, 84, 62, 19])

In [18]:
#len(vocab.get_stoi())
len(vocab)

5000

In [19]:
# ?torch.nn.Embedding

In [20]:
embedd = torch.nn.Embedding(num_embeddings=len(vocab),embedding_dim=5,padding_idx=0) 

## if we want to add embedding for a text, we should assign the value to "num_embeddings" according to the 
## max 'integer_id' that is present in the tokenized text


In [21]:
test_input = embedd(torch.tensor(vocab(tokenizer('I will see you nonsense!'))))
test_input

tensor([[-0.0644, -1.0537,  1.1503,  0.1034,  0.4832],
        [ 0.1515, -0.2933,  0.9012,  2.1108, -0.2869],
        [ 0.0197, -0.5134, -0.7790,  0.0749,  1.8912],
        [-0.0463,  0.0680,  0.9424,  0.1346,  0.2031],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [-2.1288,  0.6700,  0.9539, -1.2126, -1.2233]],
       grad_fn=<EmbeddingBackward0>)

In [22]:
test_input.shape

torch.Size([6, 5])

In [23]:
# Create the Embedding module with the correct weight matrix size
embedd = torch.nn.Embedding(len(vocab), 5, padding_idx=0)

# Check the shape of the padded_text and compare it to the expected input shape of the Embedding module
print(padded_text.shape)
# should be: torch.Size([batch_size, sequence_length])

# Use the Embedding module with the padded_text
input_text = embedd(padded_text)
print(input_text)

torch.Size([10000, 100])
tensor([[[ 0.0837,  1.5943, -0.0845,  0.4248, -1.9317],
         [-1.6009,  0.1027,  1.2657, -2.6765,  0.8783],
         [ 0.2927, -0.3056,  0.0113,  0.9458,  0.6206],
         ...,
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000]],

        [[ 0.3422, -0.6274,  0.7259, -0.8395, -0.8527],
         [ 0.8267,  0.3812, -0.2818,  0.1058, -0.1183],
         [-0.3243,  0.8092,  1.0785,  0.5918, -0.5983],
         ...,
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000]],

        [[ 0.0837,  1.5943, -0.0845,  0.4248, -1.9317],
         [-1.6009,  0.1027,  1.2657, -2.6765,  0.8783],
         [ 0.2927, -0.3056,  0.0113,  0.9458,  0.6206],
         ...,
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.00

In [24]:
# Check the shape of the padded_text and compare it to the expected input shape of the Embedding module
print(padded_text[0].shape)
# should be: torch.Size([batch_size, sequence_length])

# Use the Embedding module with the padded_text
embedd(padded_text[0]).shape


torch.Size([100])


torch.Size([100, 5])

In [25]:
df

,tweet,sentiment,label
0,im getting on borderlands and i will murder yo...,Positive,3
1,I am coming to the borders and I will kill you...,Positive,3
2,im getting on borderlands and i will kill you ...,Positive,3
3,im coming on borderlands and i will murder you...,Positive,3
4,im getting on borderlands 2 and i will murder ...,Positive,3
...,...,...,...
9995,_. . The faster more efficient AMD Zen CPUs in...,Positive,3
9996,_.. The faster more efficient AMD Zen CPUs in ...,Positive,3
9997,_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ ...,Positive,3
9998,_.. The faster memory efficient AMD Zen CPUs i...,Positive,3


In [26]:
print(padded_text.shape)
padded_text

torch.Size([10000, 100])


tensor([[ 44, 157, 238,  ...,   0,   0,   0],
        [  3, 130, 405,  ...,   0,   0,   0],
        [ 44, 157, 238,  ...,   0,   0,   0],
        ...,
        [ 51,  51,  51,  ...,  51,  51,  51],
        [ 51,  20,  39,  ...,   0,   0,   0],
        [ 51,  20,  39,  ...,   0,   0,   0]])

In [27]:
label =df['label'].to_list()

#label

In [28]:
label= torch.tensor(label)
print(label.shape)
label

torch.Size([10000])


tensor([3, 3, 3,  ..., 3, 3, 3])

In [29]:
len(label.unique())

4

In [30]:
import torch.nn as nn

# Determine the number of classes
num_classes = len(label.unique())

class RNNClassify(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_size):
        super().__init__()
        
        # Define the embedding layer
        self.embed = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        
        self.rnn = nn.RNN(embed_dim, hidden_size,batch_first=True)
        self.linear = nn.Linear(hidden_size, num_classes)
        
        # Initialize the weights of the module
        self.init_weights()
        
    def init_weights(self):
        initrange = 0.5
        self.embed.weight.data.uniform_(-initrange, initrange)
        self.rnn.weight_ih_l0.data.uniform_(-initrange, initrange)
        self.rnn.weight_hh_l0.data.uniform_(-initrange, initrange)
        self.linear.weight.data.uniform_(-initrange, initrange)
        self.linear.bias.data.zero_()
        
    def forward(self, input):
        # Embed the input
        embedded = self.embed(input)
        #print('embedded shape:',embedded.shape)
        
        # Pass the embedded input through the RNN layer
        output, hidden = self.rnn(embedded)
        #print('rnn output shape:',output.shape)
        #print('rnn hidden shape:',hidden.shape)
        
        output = output[:, -1, :]  # taking last output of RNN
        #print('rnn last output shape:',output.shape)
        
        # Pass the output through the linear layer
        output = self.linear(output)
        
        # Return the output
        return output


In [31]:
VOCAB_SIZE = len(vocab)
VOCAB_SIZE

5000

In [32]:
model = RNNClassify(vocab_size=VOCAB_SIZE,embed_dim=100,hidden_size=32).to(device)

In [33]:
padded_text[0]

tensor([  44,  157,  238,   14,   99,    7,   44,   84, 2081,   19,   41,    5,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0])

In [34]:
padded_text[0].shape

torch.Size([100])

In [35]:
padded_text[0].unsqueeze(0).shape

torch.Size([1, 100])

In [36]:
model(padded_text[0].unsqueeze(0).to(device))

tensor([[-1.1101, -1.0245, -0.6673, -2.0067]], device='cuda:0',
       grad_fn=<AddmmBackward0>)

In [37]:
model(padded_text[0].unsqueeze(0).to(device)).shape

torch.Size([1, 4])

In [38]:
padded_text

tensor([[ 44, 157, 238,  ...,   0,   0,   0],
        [  3, 130, 405,  ...,   0,   0,   0],
        [ 44, 157, 238,  ...,   0,   0,   0],
        ...,
        [ 51,  51,  51,  ...,  51,  51,  51],
        [ 51,  20,  39,  ...,   0,   0,   0],
        [ 51,  20,  39,  ...,   0,   0,   0]])

In [39]:
padded_text.shape

torch.Size([10000, 100])

In [40]:
model(padded_text.to(device))  

tensor([[-1.1101, -1.0245, -0.6673, -2.0067],
        [-1.1101, -1.0245, -0.6673, -2.0067],
        [-1.1101, -1.0245, -0.6673, -2.0067],
        ...,
        [ 1.8626, -2.0455, -0.1751, -2.3254],
        [-1.1101, -1.0245, -0.6673, -2.0067],
        [-1.1101, -1.0245, -0.6673, -2.0067]], device='cuda:0',
       grad_fn=<AddmmBackward0>)

In [41]:
model(padded_text.to(device)).shape            

torch.Size([10000, 4])

### will try `Batch Gradient Descent`

#### first of all, let me fix `(X_train,y_train)` and `(X_test,y_test)`

In [42]:
# X_train,y_train

X_train,y_train = padded_text,label

In [43]:
df_valid

,tweet,sentiment,label
0,I mentioned on Facebook that I was struggling ...,Irrelevant,0
1,BBC News - Amazon boss Jeff Bezos rejects clai...,Neutral,2
2,@Microsoft Why do I pay for WORD when it funct...,Negative,1
3,"CSGO matchmaking is so full of closet hacking,...",Negative,1
4,Now the President is slapping Americans in the...,Neutral,2
...,...,...,...
995,⭐️ Toronto is the arts and culture capital of ...,Irrelevant,0
996,tHIS IS ACTUALLY A GOOD MOVE TOT BRING MORE VI...,Irrelevant,0
997,Today sucked so it’s time to drink wine n play...,Positive,3
998,Bought a fraction of Microsoft today. Small wins.,Positive,3


In [44]:
len(df_valid)

1000

### option 1
```
valid_token_ids = []
for i in range(len(df_valid)):
    token_id = vocab(tokenizer(df_valid['tweet'][i]))
    valid_token_ids.append(token_id)
    
padded_text_valid = pad_sequence([torch.tensor(x) for x in valid_token_ids], batch_first=True, padding_value=0)

padded_text_valid = padded_text_valid[:,:MAX_LENGTH]
padded_text_valid

```

In [45]:
### option 2

# numericalize tokens from iterator using vocab for df_valid
sequence_valid = numericalize_tokens_from_iterator(vocab=vocab,iterator=token_gen(df_valid['tweet']))

# create a list to store tokenized sequences
valid_token_ids = []
for i in range(len(df_valid)):
    x = list(next(sequence_valid))
    valid_token_ids.append(x)
    
# valid_token_ids = torch.tensor(valid_token_ids) # this will throw an error, because all sequence are not of same length

# Pad the sequences to the same length along dimension 0
padded_text_valid = pad_sequence([torch.tensor(x) for x in valid_token_ids], batch_first=True, padding_value=0)

# restrict the length of every sequence upto MAX_LENGTH
padded_text_valid = padded_text_valid[:,:MAX_LENGTH]

print(padded_text_valid.shape)
print(padded_text_valid)

torch.Size([1000, 70])
tensor([[   3,    0,   14,  ...,    0,    0,    0],
        [1805, 1074,   21,  ...,    0,    0,    0],
        [   0,  259,   45,  ...,    0,    0,    0],
        ...,
        [ 744, 1492,   28,  ...,    0,    0,    0],
        [2542,    8,    0,  ...,    0,    0,    0],
        [4357,  201, 4357,  ...,    0,    0,    0]])


In [46]:
label_valid = df_valid['label'].to_list()

In [47]:
label_valid = torch.tensor(label_valid)
#label_valid

In [48]:
# X_test,y_test

X_test, y_test = padded_text_valid, label_valid
X_test, y_test

(tensor([[   3,    0,   14,  ...,    0,    0,    0],
         [1805, 1074,   21,  ...,    0,    0,    0],
         [   0,  259,   45,  ...,    0,    0,    0],
         ...,
         [ 744, 1492,   28,  ...,    0,    0,    0],
         [2542,    8,    0,  ...,    0,    0,    0],
         [4357,  201, 4357,  ...,    0,    0,    0]]),
 tensor([0, 2, 1, 1, 2, 1, 3, 3, 3, 1, 3, 3, 1, 2, 1, 3, 3, 1, 3, 1, 1, 2, 0, 1,
         2, 2, 1, 0, 0, 1, 3, 3, 1, 3, 1, 2, 2, 0, 3, 2, 3, 2, 2, 2, 3, 2, 1, 1,
         1, 2, 3, 1, 1, 3, 3, 3, 3, 3, 1, 0, 1, 3, 3, 0, 1, 2, 1, 0, 2, 1, 3, 1,
         1, 3, 3, 0, 3, 0, 2, 2, 2, 3, 3, 2, 3, 2, 1, 0, 1, 2, 2, 1, 3, 0, 0, 1,
         1, 1, 2, 3, 2, 1, 3, 3, 2, 3, 2, 3, 1, 2, 2, 2, 1, 2, 1, 2, 2, 3, 3, 2,
         1, 1, 3, 1, 2, 1, 3, 2, 1, 2, 0, 3, 2, 3, 3, 0, 2, 2, 0, 0, 0, 2, 2, 0,
         0, 0, 3, 2, 3, 0, 3, 1, 2, 2, 2, 0, 2, 1, 2, 3, 1, 2, 1, 0, 0, 0, 2, 1,
         1, 1, 3, 3, 3, 2, 2, 3, 0, 2, 2, 2, 3, 2, 1, 1, 2, 3, 3, 0, 0, 2, 3, 3,
         2, 0, 2, 

#### Now, write the train and test loop for `Batch Gradient Descent`

In [49]:
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
loss_fn = torch.nn.CrossEntropyLoss()  # remember it gives logits (row outputs)

In [50]:
# Here, we will use Batch Gradient Descent, BUT, generally we prefer Mini-Batch Gradient Descent

epochs = 30


for epoch in range(epochs):
    train_loss,train_acc = 0,0
    
    # Set model to training mode
    model.train()
    
    X_train,y_train = X_train.to(device), y_train.to(device)
    
    y_logits = model(X_train)
    #print('shape of y_logits:',y_logits.shape)

    
    # Compute loss with one-hot encoded targets
    loss = loss_fn(y_logits, y_train)
    
    train_loss += loss
    train_acc += (y_logits.argmax(1) == y_train).sum().item() / len(y_train)
    
    optimizer.zero_grad()
    loss.backward()
    
    optimizer.step()
    
    print(f'Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.4f}')
    
    
    with torch.inference_mode():
        
        model.eval()
        
        test_loss,test_acc = 0,0
    
        X_test, y_test = X_test.to(device), y_test.to(device)
        
        y_logits = model(X_test)

        # Compute loss with one-hot encoded targets
        loss = loss_fn(y_logits, y_test)

        test_loss += loss.item()
            
        # Compute accuracy
        test_preds = y_logits.argmax(dim=1)
        test_acc += (test_preds == y_test).sum().item() / len(y_test)

        
        print(f'Epoch {epoch+1}/{epochs}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}')
        
        print('--'*50)

Epoch 1/30, Train Loss: 1.5519, Train Accuracy: 0.2618
Epoch 1/30, Test Loss: 1.4597, Test Accuracy: 0.2850
----------------------------------------------------------------------------------------------------
Epoch 2/30, Train Loss: 1.5052, Train Accuracy: 0.2618
Epoch 2/30, Test Loss: 1.4303, Test Accuracy: 0.2840
----------------------------------------------------------------------------------------------------
Epoch 3/30, Train Loss: 1.4708, Train Accuracy: 0.2617
Epoch 3/30, Test Loss: 1.4078, Test Accuracy: 0.2840
----------------------------------------------------------------------------------------------------
Epoch 4/30, Train Loss: 1.4422, Train Accuracy: 0.2617
Epoch 4/30, Test Loss: 1.3909, Test Accuracy: 0.2810
----------------------------------------------------------------------------------------------------
Epoch 5/30, Train Loss: 1.4183, Train Accuracy: 0.2618
Epoch 5/30, Test Loss: 1.3798, Test Accuracy: 0.2800
--------------------------------------------------------